In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(
    "../../data/SP500_EGS_Score_avarage_per_year.csv",
    index_col=0,
    parse_dates=True,
    sep=",",
)

In [ ]:
df.head(20)

In [ ]:
# all lower case

df.columns = map(str.lower, df.columns)

# - to _

df.columns = df.columns.str.replace("-", "_")

In [ ]:
len(df.company_symbol.unique())

In [ ]:
import sys

sys.path.append("../")

In [ ]:
from helpers.text_preprocessing.preprocess_text import preprocess_text

In [ ]:
import pandas as pd

# Load or create your DataFrame (replace 'your_data.csv' with the path to your data file)
df = pd.read_csv(
    "../data/extracted_text_sustainability_reports.csv",
    index_col=0,
    parse_dates=True,
    sep=",",
)

In [ ]:
# Preprocess the text and get the preprocessed DataFrame
preprocessed_df = preprocess_text(df)

In [ ]:
import optuna
import pandas as pd
import matplotlib.pyplot as plt
import plotly
from gensim import corpora
import mlflow

In [ ]:
import sys

sys.path.append("../")

In [ ]:
from models.LDA_optuna_tuning.tune_lda_optuna import train_lda, compute_coherence
from models.LDA_optuna_tuning.call_optuna_tune import (
    preprocess_data,
    execute_optuna_study,
)

In [1]:
from sklearn.model_selection import train_test_split, KFold

import tpot2
import sklearn
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../data/ready_to_model/df_filtered_feature_importance.csv")


In [ ]:
df.fillna(0, inplace=True)

In [3]:
'''"e_score",
"s_score",
"g_score",
"unnamed: 0",
"filename",
"ticker",
"year",
"preprocessed_content",
"ner_entities",
"company_symbol",'''

# columns to drop
columns_to_drop = [

    "total_score",
    "Unnamed: 0"
]

# Separate features and target
y = df["total_score"]
X = df.drop(columns=columns_to_drop)

In [4]:
# drop the last two rows
X = X.iloc[:-2, :]

In [5]:
y = y.iloc[:-2]

In [ ]:
X

In [ ]:
X

In [ ]:
y

In [6]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=100
)

In [ ]:
scorer = sklearn.metrics.get_scorer("neg_mean_squared_error")

# Initialize TPOT2 regressor with K-Fold cross-validation
est = tpot2.TPOTEstimatorSteadyState(
    n_jobs=6,
    cv=KFold(n_splits=5),  # 5-Fold cross-validation
    verbose=2,
    classification=False,
    scorers=[scorer],
    scorers_weights=[1],
    max_eval_time_seconds=60 * 10,
    max_time_seconds=60 * 90,
)

# Fit the model
est.fit(X_train, y_train)
print('Done fitting/training TPOT2 session.')


df_individuals = est.evaluated_individuals

# Convert the 'mean_squared_error' column to numeric, errors='coerce' will replace non-numeric with NaN
df_individuals['mean_squared_error'] = pd.to_numeric(df_individuals['mean_squared_error'], errors='coerce')

# Drop NaN values
filtered_df = df_individuals.dropna(subset=['mean_squared_error'])

# Sort the DataFrame by 'mean_squared_error' and get the top 10
top_10_mse = filtered_df.nlargest(10, 'mean_squared_error')

print(est.pareto_front)
print(top_10_mse)
print(est.get_params())



In [ ]:
top_10_mse

In [ ]:
top_30_mse = filtered_df.nlargest(30, 'mean_squared_error')


In [ ]:
# save the top 30 models to csv
top_30_mse.to_csv("../data/model_data/top_30_mse.csv")

In [ ]:
top_30_mse

In [7]:
import sys

sys.path.append("../")

In [8]:
from models.XGBoost.train_and_evaluate_model import train_and_evaluate_model
from models.XGBoost.tune_xgb_hyperparameters import tune_xgb_hyperparameters
from models.Random_Forest.tune_rf_hyperparameters import tune_rf_hyperparameters
from models.Lasso.tune_lasso_hyperparameters import tune_lasso_hyperparameters
from models.Neural_Network.tune_nn_hyperparameters import tune_nn_hyperparameters
from models.Ridge.tune_ridge_hyperparameters import tune_ridge_hyperparameters
from models.perform_stacking import perform_stacking
from models.optimize_stacking import optimize_stacking

In [ ]:
print(X_train.dtypes)


In [ ]:
X_train = X_train.apply(pd.to_numeric, errors='ignore')


In [ ]:
best_params_rf = tune_rf_hyperparameters(X_train, y_train, 25)

In [ ]:
#Best hyperparameters: {'n_estimators': 266, 'max_depth': 32, 'min_samples_split': 4, 'min_samples_leaf': 4, 'max_features': None}
# Best RMSE: 5.883527888313719

In [ ]:
best_params_lasso = tune_lasso_hyperparameters(X_train, y_train, X_test, y_test, 100)

In [ ]:
# [I 2023-10-04 17:44:22,225] Trial 23 finished with value: 4.497792051287519 and parameters: {'alpha': 0.08706251396825564}. Best is trial 23 with value: 4.497792051287519.
# Training RMSE: 5.277817560823488, Test RMSE: 4.497792051287519

In [ ]:
best_params_ridge = tune_ridge_hyperparameters(X_train, y_train, X_test, y_test, 100)

In [ ]:
# Best hyperparameters: {'alpha': 0.9992235606570956}
# Best Test RMSE: 5.210845073517565

In [ ]:
best_params_nn = tune_nn_hyperparameters(X_train, y_train, X_test, y_test, 30)

In [ ]:
# Best hyperparameters: {'hidden_layer_sizes': (50, 50), 'activation': 'tanh', 'alpha': 0.00010434024177879637}
# Best Test RMSE: 6.9895163966718235

In [ ]:
best_params = tune_xgb_hyperparameters(X_train, y_train, n_trials=30)


In [ ]:
# Best hyperparameters: {'learning_rate': 0.02998822459568964, 'max_depth': 5, 'subsample': 0.7095254982521659, 'colsample_bytree': 0.6113315193397809, 'min_child_weight': 13}
# Best RMSE: 5.098858708449277

In [9]:
import pickle

import warnings
warnings.filterwarnings("ignore")


In [ ]:
with open('../models/XGBoost/best_params_features_cleaned.pkl', 'rb') as f:
    best_params_xgb = pickle.load(f)

with open('../models/Lasso/best_params_lasso.pkl', 'rb') as f:
    best_params_lasso = pickle.load(f)

with open('../models/Ridge/best_params_ridge.pkl', 'rb') as f:
    best_params_ridge = pickle.load(f)
    
with open('../models/Random_Forest/best_params_rf.pkl', 'rb') as f:
    best_params_rf = pickle.load(f)

with open('../models/Neural_Network/best_params_nn.pkl', 'rb') as f:
    best_params_nn = pickle.load(f)

In [ ]:
best_params = optimize_stacking(X, y, n_trials=25)


In [ ]:
stacking_model, test_rmse = perform_stacking(X, y, best_params_lasso, best_params_rf, best_params_xgb)

In [ ]:
# STACKING MODEL
# base models: ['lasso', 'lasso_lars_cv', 'xgb', 'rf', 'ridge_cv']
# meta model: gradient_boosting_regressor

In [ ]:
# VOTING REGRESSOR
# Root Mean Squared Error (RMSE) for test set: 5.170260299240855
# Root Mean Squared Error (RMSE) for training set: 3.3442587249065046

# ADABOOST REGRESSOR
# Root Mean Squared Error (RMSE) for test set: 4.644395241213758
# Root Mean Squared Error (RMSE) for training set: 2.0869823753568713

# GRADIENT BOOSTING REGRESSOR
# Root Mean Squared Error (RMSE) for test set: 4.496692472551547
# Root Mean Squared Error (RMSE) for training set: 1.653435201586618

# ADA BOOST REGRESSOR
# Root Mean Squared Error (RMSE) for test set: 5.401425699939552
# Root Mean Squared Error (RMSE) for training set: 4.3236806588367855

# LGBM REGRESSOR
# Root Mean Squared Error (RMSE) for test set: 4.546673980636338
# Root Mean Squared Error (RMSE) for training set: 0.2985705242417246

In [11]:
# open pickle file from the best_params_xgb
with open('../models/XGBoost/best_params_features_cleaned.pkl', 'rb') as f:
    best_params = pickle.load(f)

In [12]:
bagging_model, rmse_val, feature_importance_df = train_and_evaluate_model(X_train, y_train, X_test, y_test, best_params)

RMSE on Train set: 3.37877768036756
Feature Importances:
                                               Feature  Importance
0                                      tfidf_svd_dim_4    0.011436
298                                                284    0.010542
3                                                   69    0.010370
335                                                420    0.008674
52   financials_Net Non Operating Interest Income E...    0.007855
..                                                 ...         ...
527                                                625    0.000000
185                                         svd_dim_65    0.000000
522                                                546    0.000000
516                                                377    0.000000
399                                                698    0.000000

[798 rows x 2 columns]
RMSE on Validation set: 4.988712767980542


In [14]:
feature_importance_df

,Feature,Importance
0,tfidf_svd_dim_4,0.011436
298,284,0.010542
3,69,0.010370
335,420,0.008674
52,financials_Net Non Operating Interest Income E...,0.007855
...,...,...
527,625,0.000000
185,svd_dim_65,0.000000
522,546,0.000000
516,377,0.000000


In [17]:
filtered_feature_importance_df = feature_importance_df[feature_importance_df['Importance'] != 0]


In [18]:
df

,Unnamed: 0,tfidf_svd_dim_4,637,134,69,287,83,38,balance_Additional Paid In Capital,balance_Net PPE,...,353,svd_dim_14,svd_dim_101,32,cashflow_Long Term Debt Issuance,balance_Accounts Payable,balance_Current Debt,695,svd_dim_50,total_score
0,0,-0.084914,-0.984096,0.268918,0.107418,-0.046604,-1.011947,-0.375769,0.000000e+00,2.815400e+09,...,-0.384085,-0.382449,0.050529,-0.734844,1.000000e+09,6.213000e+08,1500000.0,0.935146,-0.036939,15.30
1,1,0.011697,-0.408780,0.058946,-0.211146,-0.258639,-1.149698,0.105433,6.617404e+09,1.161558e+09,...,0.259594,0.004463,-0.062224,-0.494972,0.000000e+00,2.166409e+09,59482000.0,0.983424,0.011989,27.66
2,2,-0.045409,0.379066,0.062867,-0.268551,-0.470384,-0.945055,1.394683,3.160000e+06,9.470280e+08,...,-0.082250,-0.999183,0.172444,-0.779211,NaN,2.012461e+09,NaN,0.749328,-0.801159,18.13
3,3,0.057648,-0.725314,0.736935,-0.064104,-0.223336,-0.799952,0.159469,NaN,NaN,...,-1.051099,-0.039436,-0.029329,-0.679992,NaN,NaN,NaN,0.727001,0.598416,18.25
4,4,0.012955,-0.828527,-0.378939,-0.294139,0.299100,-1.169436,-0.405682,1.254200e+09,6.876000e+08,...,1.061694,1.390666,-0.036480,-0.481583,9.894000e+08,1.484000e+08,NaN,0.349011,-1.085175,21.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,844,0.026311,-0.129050,0.170490,-0.054193,-0.399259,-0.675335,0.057946,9.537000e+09,NaN,...,-0.103423,-0.755071,-0.001880,-0.450927,5.330000e+09,NaN,NaN,0.514164,-0.469090,21.70
845,845,-0.015391,-0.068218,0.163940,0.009989,-0.350510,-1.147584,0.601176,1.581000e+09,1.837400e+10,...,-0.156433,-0.379208,-0.044857,-0.130985,0.000000e+00,1.574000e+09,819000000.0,1.275327,0.384824,30.41
846,846,0.035103,-0.597744,-0.269980,0.086464,-0.059405,-0.315838,0.366545,NaN,NaN,...,-0.020172,0.668182,0.141643,-0.240015,NaN,NaN,NaN,0.807168,0.151807,19.73
847,847,-0.002000,-0.430859,-0.107497,0.018429,-0.108159,-0.717960,0.478447,1.434203e+09,2.342330e+08,...,-0.025405,0.027800,0.109576,-0.545853,3.750000e+08,1.869100e+07,NaN,1.088519,0.030471,15.40


In [19]:
filtered_feature_importance_df

,Feature,Importance
0,tfidf_svd_dim_4,0.011436
298,284,0.010542
3,69,0.010370
335,420,0.008674
52,financials_Net Non Operating Interest Income E...,0.007855
...,...,...
219,tfidf_svd_dim_98,0.000188
421,svd_dim_111,0.000184
657,387,0.000176
722,736,0.000147


In [20]:
filtered_main_df = df.loc[:, filtered_feature_importance_df['Feature'].tolist() + ['total_score']]



In [21]:
filtered_main_df

,tfidf_svd_dim_4,284,69,420,financials_Net Non Operating Interest Income Expense,220,717,balance_Net PPE,134,balance_Non Current Deferred Liabilities,...,svd_dim_10,254,631,184,tfidf_svd_dim_98,svd_dim_111,387,736,tfidf_svd_dim_48,total_score
0,-0.084914,-1.045438,0.107418,0.024851,-212100000.0,1.326941,0.477323,2.815400e+09,0.268918,7.634000e+08,...,0.386335,1.438630,-0.235152,0.016359,-0.018976,0.119630,-0.275476,0.156166,-0.000129,15.30
1,0.011697,-0.338321,-0.211146,-0.468352,NaN,2.095232,0.356844,1.161558e+09,0.058946,NaN,...,-1.506424,1.581487,-0.086529,0.036891,0.033138,-0.066276,0.161144,0.196399,-0.032492,27.66
2,-0.045409,0.255509,-0.268551,0.305231,8807000.0,-0.075387,0.452430,9.470280e+08,0.062867,0.000000e+00,...,3.641058,2.048474,-0.025404,-0.710708,0.041429,-0.440183,-0.342687,-0.472374,-0.047119,18.13
3,0.057648,-0.396426,-0.064104,-0.044723,NaN,0.926291,0.574815,NaN,0.736935,NaN,...,0.939981,1.391185,0.034777,-0.015660,-0.005421,0.045432,-0.691693,0.295962,-0.096748,18.25
4,0.012955,-0.707049,-0.294139,1.144451,-10400000.0,0.560736,0.043149,6.876000e+08,-0.378939,1.675500e+09,...,2.451167,0.637493,-0.558616,-0.181851,-0.095714,0.407011,-0.684295,1.446273,-0.047031,21.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,0.026311,-0.480919,-0.054193,-0.876421,NaN,1.338712,0.520521,NaN,0.170490,NaN,...,-0.363391,1.752549,0.279902,-0.481525,0.030541,0.104953,-0.079488,0.749023,-0.001181,21.70
845,-0.015391,-0.346565,0.009989,-0.546589,8000000.0,1.665551,0.441377,1.837400e+10,0.163940,3.417000e+09,...,-0.903421,1.747730,-0.225536,-0.076029,-0.077112,0.261840,0.131072,-0.361655,-0.083406,30.41
846,0.035103,-0.776896,0.086464,-0.405729,NaN,2.058354,0.085975,NaN,-0.269980,NaN,...,-1.809347,1.397504,-0.463194,0.123010,0.021314,-0.007611,0.231459,-0.853759,-0.009270,19.73
847,-0.002000,-0.771184,0.018429,0.253053,-5915000.0,1.785508,0.439379,2.342330e+08,-0.107497,1.103210e+08,...,0.548983,1.481942,-0.157322,-0.015291,0.018659,0.247267,-0.340734,-0.022198,0.072752,15.40


In [ ]:
filtered_main_df.to_csv("../data/ready_to_model/filtered_feature_importance_df.csv")

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import yfinance as yf
import pandas as pd


def fetch_data_for_row(named_tuple_row):
    ticker = "Unknown"  # Initialize with a default value
    year = "Unknown"    # Initialize with a default value
    
    try:
        ticker = named_tuple_row.ticker
        year = named_tuple_row.year

        yf_ticker = yf.Ticker(ticker)
        financials = yf_ticker.financials
        cashflow = yf_ticker.cashflow
        balance = yf_ticker.balance_sheet
        info = {k: v for k, v in yf_ticker.info.items() if isinstance(v, (int, float))}

        financials = financials.loc[:, pd.to_datetime(financials.columns).year == year].transpose()
        cashflow = cashflow.loc[:, pd.to_datetime(cashflow.columns).year == year].transpose()
        balance = balance.loc[:, pd.to_datetime(balance.columns).year == year].transpose()
        info_df = pd.DataFrame([info])

        financials.columns = 'financials_' + financials.columns.astype(str)
        cashflow.columns = 'cashflow_' + cashflow.columns.astype(str)
        balance.columns = 'balance_' + balance.columns.astype(str)
        info_df.columns = 'info_' + info_df.columns.astype(str)

        merged_data = pd.concat([financials, cashflow, balance, info_df], axis=1)
        merged_data['ticker'] = ticker
        merged_data['year'] = year

        return merged_data.reset_index(drop=True)
    except Exception as e:
        print(f"An error occurred in fetch_data_for_row for ticker: {ticker} and year: {year}. Error: {e}")
        return pd.DataFrame()



def fetch_and_merge_data(df):
    try:
        # Initialize an empty list to store fetched data
        fetched_data_list = []
        
        with ThreadPoolExecutor() as executor:
            fetched_data_list = list(executor.map(fetch_data_for_row, df.itertuples(index=False)))

        # Concatenate all the fetched data
        new_data = pd.concat([data.iloc[[0]] for data in fetched_data_list if not data.empty], ignore_index=True)

        # Debug: Print the shape and columns of new_data
        print(f"new_data shape: {new_data.shape}, columns: {new_data.columns}")

        # Merge new_data with df based on 'ticker' and 'year'
        final_df = pd.merge(df, new_data, on=['ticker', 'year'], how='left')

        return final_df
    except Exception as e:
        print(f"An error occurred in fetch_and_merge_data: {e}")
        return df  # Return the original DataFrame as a fallback




# Example usage
# df = pd.DataFrame({'ticker': ['AAPL', 'GOOGL'], 'year': [2020, 2021]})
# final_df = fetch_and_merge_data(df)


In [ ]:
df

In [ ]:
final_df = fetch_and_merge_data(df)

In [ ]:
df = final_df

In [ ]:
final_df.sample(10)

In [ ]:
# Calculate the percentage of missing values for each column
missing_percent = df_cleaned.isnull().mean() * 100

# Sort the columns by percentage of missing values in descending order
missing_percent_sorted = missing_percent.sort_values(ascending=False)

# Show the sorted series
print(missing_percent_sorted)


In [ ]:
nan_count = final_df.isna().sum()


In [ ]:
def remove_columns_with_nans(df, threshold=800):
    nan_count = final_df.isna().sum()
    columns_to_remove = nan_count[nan_count > threshold].index.tolist()
    df_cleaned = final_df.drop(columns=columns_to_remove)
    return df_cleaned

In [ ]:
df_cleaned = remove_columns_with_nans(df, threshold=300)


In [ ]:
df_cleaned



In [ ]:
df_cleaned.to_csv("../data/ready_to_model/df_cleaned_with_yfinance.csv")

In [ ]:
# Sorting columns by the number of NaN values (in descending order)
sorted_nan_count = nan_count.sort_values(ascending=False)

print(sorted_nan_count.value_counts())

In [ ]:
import pandas as pd

In [ ]:
df_topics = pd.read_csv("../data/ready_to_model/df_cleaned_with_yfinance.csv", index_col=0, parse_dates=True, sep=",")

In [ ]:
df_topics.head()